In [1]:
cd ..

/Users/n.barsukov/PycharmProjects/toxic-comments-detector


In [2]:
import sys
import tensorflow as tf
import numpy as np

# local (this repo) imports
from dev import load_cleaned_russian_text_data, SEED

tf.random.set_seed(SEED)

print('Python version:', sys.version)
print('Tensorflow version:', tf.__version__)

Python version: 3.8.9 (default, Apr  3 2021, 01:50:09) 
[Clang 12.0.0 (clang-1200.0.32.29)]
Tensorflow version: 2.4.1


## Load data

In [3]:
X_train, X_test, y_train, y_test = load_cleaned_russian_text_data()

In [4]:
def get_3d_quartile_words_count(sentences_array):
    """
    Получает массив строк/предложений.
    Отдает число N.
    75% всех предложений данного массива не превышают N слов в длину.
    """
    def count_words(sentence):
        return len(sentence.split(' '))
        
    
    return round(
        np.percentile(
            list(map(count_words, sentences_array)),
            75
        )
    )

## Train model

In [5]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers, losses

In [6]:
MAX_FEATURES = 10000
MAX_SEQUENCE_LENGTH = get_3d_quartile_words_count(X_train.flatten())

vectorize_layer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)
vectorize_layer.adapt(X_train)

In [7]:
model = tf.keras.Sequential([
    layers.Input(shape=[], dtype=tf.string),
    vectorize_layer,
    layers.Embedding(
        input_dim = len(vectorize_layer.get_vocabulary()),
        output_dim = 16
    ),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1),
    layers.Activation('sigmoid') # то есть в конце мы выдаем уже вероятности
])

model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)

In [8]:
epochs = 10
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    batch_size=2**7,
    validation_split=0.2, # сколько от тестовой выборки отрезать под валидационную
)

Epoch 1/10
73/73 [==============================] - 2s 14ms/step - loss: 0.6806 - binary_accuracy: 0.6235 - recall: 0.1069 - precision: 0.3640 - val_loss: 0.6458 - val_binary_accuracy: 0.6670 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
73/73 [==============================] - 0s 3ms/step - loss: 0.6382 - binary_accuracy: 0.6604 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.6142 - val_binary_accuracy: 0.6670 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/10
73/73 [==============================] - 0s 3ms/step - loss: 0.6052 - binary_accuracy: 0.6659 - recall: 1.1015e-04 - precision: 0.2973 - val_loss: 0.5862 - val_binary_accuracy: 0.6670 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
73/73 [==============================] - 0s 3ms/step - loss: 0.5795 - binary_accuracy: 0.6593 - recall: 0.0061 - precision: 0.9595 - val_loss: 0.5613 - val_binary_accuracy: 0.6748 - val_recall: 0.0260 - val_precision: 0.9091
Epoch 5/10
73/73 [=

In [9]:
loss, accuracy, recall, precision = model.evaluate(x=X_test, y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("Precision: ", precision)

91/91 [==============================] - 0s 761us/step - loss: 0.4353 - binary_accuracy: 0.8456 - recall: 0.6226 - precision: 0.8748
Loss:  0.4352737367153168
Accuracy:  0.8456469178199768
Recall:  0.6226415038108826
Precision:  0.8748158812522888


In [10]:
tf.math.confusion_matrix(labels=y_test, predictions=(model.predict(X_test) > 0.5) * 1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1844,   85],
       [ 360,  594]], dtype=int32)>